# ETL: Silver para Gold (Data Warehouse)

Este notebook realiza a transformação dos dados da camada **Silver** para a camada **Gold** (Data Warehouse).

## Estrutura de Transformação

**Silver (Star Schema):**
- `silver.dim_modelo` - Dimensão de modelos
- `silver.dim_especificacao` - Dimensão de especificações
- `silver.fato_veiculo` - Tabela fato

**Gold (Data Warehouse - Schema `dw`):**
- `dw.DIM_MODELO` - Dimensão de modelos (normalizada)
- `dw.DIM_CONDICAO` - Dimensão de condições (seller_type, condition, accident_history)
- `dw.DIM_COR` - Dimensão de cores (exterior_color, interior_color)
- `dw.FATO_VEICULO` - Tabela fato com chaves estrangeiras (SRK)

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime
import os

## 1. Configuração e Conexão


In [3]:
# Configuração do banco de dados
DB_CONFIG = {
    'host': os.getenv('POSTGRES_HOST', 'postgres'),
    'port': os.getenv('POSTGRES_PORT', '5432'),
    'database': os.getenv('POSTGRES_DB', 'sbd2_vehicle'),
    'user': os.getenv('POSTGRES_USER', 'sbd2_vehicle'),
    'password': os.getenv('POSTGRES_PASSWORD', 'sbd2_vehicle')
}

# Criar engine SQLAlchemy
connection_string = (
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}"
    f"@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)
engine = create_engine(connection_string)

print("Conexão estabelecida com sucesso!")
print(f"Host: {DB_CONFIG['host']}")
print(f"Database: {DB_CONFIG['database']}")


Conexão estabelecida com sucesso!
Host: postgres
Database: sbd2_vehicle


## 2. Criar Schema e Tabelas Gold (Data Warehouse)


In [4]:
# Ler o arquivo DDL
ddl_path = '../DataLayer/gold/ddl_gold.sql'

print("Criando schema e tabelas Gold (dw)...")
print("="*60)

try:
    with open(ddl_path, 'r') as f:
        ddl_content = f.read()
    
    # Executar o DDL
    with engine.connect() as conn:
        # Executar cada statement do DDL
        for statement in ddl_content.split(';'):
            statement = statement.strip()
            if statement:
                try:
                    conn.execute(text(statement))
                    conn.commit()
                except Exception as e:
                    # Ignorar erros de "já existe" ou "não existe"
                    if 'already exists' not in str(e).lower() and 'does not exist' not in str(e).lower():
                        print(f"  Aviso: {e}")
    
    print("  Schema 'dw' criado")
    print("  Tabelas Gold criadas:")
    print("    - dw.DIM_MODELO")
    print("    - dw.DIM_CONDICAO")
    print("    - dw.DIM_COR")
    print("    - dw.FATO_VEICULO")
    
except Exception as e:
    print(f"  Erro ao criar tabelas: {e}")
    raise


Criando schema e tabelas Gold (dw)...
  Schema 'dw' criado
  Tabelas Gold criadas:
    - dw.DIM_MODELO
    - dw.DIM_CONDICAO
    - dw.DIM_COR
    - dw.FATO_VEICULO


## 3. Extrair Dados da Camada Silver


In [5]:
print("Extraindo dados da camada Silver...")
print("="*60)

# Ler dados das tabelas Silver
print("  Lendo silver.dim_modelo...")
df_dim_modelo_silver = pd.read_sql("SELECT * FROM silver.dim_modelo", engine)
print(f"    {len(df_dim_modelo_silver):,} registros")

print("  Lendo silver.dim_especificacao...")
df_dim_espec_silver = pd.read_sql("SELECT * FROM silver.dim_especificacao", engine)
print(f"    {len(df_dim_espec_silver):,} registros")

print("  Lendo silver.fato_veiculo...")
df_fato_silver = pd.read_sql("SELECT * FROM silver.fato_veiculo", engine)
print(f"    {len(df_fato_silver):,} registros")

print("\n  Dados extraídos com sucesso!")


Extraindo dados da camada Silver...
  Lendo silver.dim_modelo...
    981,719 registros
  Lendo silver.dim_especificacao...
    29,671 registros
  Lendo silver.fato_veiculo...
    1,000,000 registros

  Dados extraídos com sucesso!


## 4. Transformar e Criar Dimensões Gold


In [6]:
print("Transformando dimensões...")
print("="*60)

# DIM_MODELO (Gold)
print("\n[1/3] Criando dw.DIM_MODELO...")

# Selecionar colunas da dimensão modelo
dim_modelo_cols = [
    'make', 'model', 'year', 'trim', 'engine_hp',
    'transmission', 'fuel_type', 'drivetrain', 'body_type'
]

# Criar DataFrame com combinações únicas
df_dim_modelo_gold = df_dim_modelo_silver[dim_modelo_cols].drop_duplicates().reset_index(drop=True)

print(f"  {len(df_dim_modelo_gold):,} modelos únicos")

# DIM_CONDICAO (Gold)
print("\n[2/3] Criando dw.DIM_CONDICAO...")

# Selecionar colunas da dimensão condição
dim_condicao_cols = [
    'seller_type', 'condition', 'accident_history'
]

# Criar DataFrame com combinações únicas
df_dim_condicao_gold = df_dim_espec_silver[dim_condicao_cols].drop_duplicates().reset_index(drop=True)

print(f"  {len(df_dim_condicao_gold):,} condições únicas")

# DIM_COR (Gold)
print("\n[3/3] Criando dw.DIM_COR...")

# Selecionar colunas da dimensão cor
dim_cor_cols = [
    'exterior_color', 'interior_color'
]

# Criar DataFrame com combinações únicas
df_dim_cor_gold = df_dim_espec_silver[dim_cor_cols].drop_duplicates().reset_index(drop=True)

print(f"  {len(df_dim_cor_gold):,} combinações de cores únicas")

print("\n  Dimensões transformadas com sucesso!")


Transformando dimensões...

[1/3] Criando dw.DIM_MODELO...
  981,719 modelos únicos

[2/3] Criando dw.DIM_CONDICAO...
  18 condições únicas

[3/3] Criando dw.DIM_COR...
  24 combinações de cores únicas

  Dimensões transformadas com sucesso!


## 5. Carregar Dimensões no Data Warehouse


In [7]:
print("Carregando dimensões no Data Warehouse...")
print("="*60)

start_time = datetime.now()

# Verificar se já existem dados e limpar se necessário
with engine.connect() as conn:
    try:
        result = conn.execute(text("SELECT COUNT(*) FROM dw.FATO_VEICULO"))
        count = result.scalar()
        if count > 0:
            print(f"  ⚠️  Gold já contém {count:,} registros. Limpando tabelas...")
            # Limpar tabelas (ordem importante devido às FKs)
            conn.execute(text("TRUNCATE TABLE dw.FATO_VEICULO CASCADE"))
            conn.execute(text("TRUNCATE TABLE dw.DIM_MODELO CASCADE"))
            conn.execute(text("TRUNCATE TABLE dw.DIM_CONDICAO CASCADE"))
            conn.execute(text("TRUNCATE TABLE dw.DIM_COR CASCADE"))
            conn.commit()
            print("  ✓ Tabelas limpas")
    except Exception as e:
        # Se a tabela não existir ainda, não há problema
        if 'does not exist' not in str(e).lower():
            print(f"  Aviso: {e}")

# Carregar DIM_MODELO
print("\n  Carregando dw.DIM_MODELO...")
df_dim_modelo_gold.to_sql(
    'DIM_MODELO',
    engine,
    schema='dw',
    if_exists='append',
    index=False,
    chunksize=5000
)
print(f"    {len(df_dim_modelo_gold):,} registros inseridos")

# Carregar DIM_CONDICAO
print("  Carregando dw.DIM_CONDICAO...")
df_dim_condicao_gold.to_sql(
    'DIM_CONDICAO',
    engine,
    schema='dw',
    if_exists='append',
    index=False,
    chunksize=5000
)
print(f"    {len(df_dim_condicao_gold):,} registros inseridos")

# Carregar DIM_COR
print("  Carregando dw.DIM_COR...")
df_dim_cor_gold.to_sql(
    'DIM_COR',
    engine,
    schema='dw',
    if_exists='append',
    index=False,
    chunksize=5000
)
print(f"    {len(df_dim_cor_gold):,} registros inseridos")

duration = (datetime.now() - start_time).total_seconds()
print(f"\n  Dimensões carregadas em {duration:.1f}s")


Carregando dimensões no Data Warehouse...

  Carregando dw.DIM_MODELO...
    981,719 registros inseridos
  Carregando dw.DIM_CONDICAO...
    18 registros inseridos
  Carregando dw.DIM_COR...
    24 registros inseridos

  Dimensões carregadas em 39.5s


## 6. Ler Dimensões Gold com SRKs Gerados


In [12]:
print("Lendo dimensões Gold com SRKs gerados...")
print("="*60)

# Ler dimensões do banco para obter os SRKs gerados
df_dim_modelo_gold_db = pd.read_sql("SELECT * FROM dw.DIM_MODELO", engine)
df_dim_condicao_gold_db = pd.read_sql("SELECT * FROM dw.DIM_CONDICAO", engine)
df_dim_cor_gold_db = pd.read_sql("SELECT * FROM dw.DIM_COR", engine)

print(f"  dw.DIM_MODELO: {len(df_dim_modelo_gold_db):,} registros")
print(f"  dw.DIM_CONDICAO: {len(df_dim_condicao_gold_db):,} registros")
print(f"  dw.DIM_COR: {len(df_dim_cor_gold_db):,} registros")

print("\n  Dimensões lidas com sucesso!")


Lendo dimensões Gold com SRKs gerados...
  dw.DIM_MODELO: 0 registros
  dw.DIM_CONDICAO: 0 registros
  dw.DIM_COR: 0 registros

  Dimensões lidas com sucesso!


## 7. Transformar e Criar Tabela Fato Gold


In [14]:
print("Transformando tabela fato...")
print("="*60)

# Fazer join do fato silver com as dimensões silver para obter os dados completos
print("  Fazendo join com dimensões Silver...")

# Join fato com dim_modelo
df_fato_completo = df_fato_silver.merge(
    df_dim_modelo_silver,
    on='id_modelo',
    how='left'
)

# Join fato com dim_especificacao
df_fato_completo = df_fato_completo.merge(
    df_dim_espec_silver,
    on='id_especificacao',
    how='left'
)

print(f"  {len(df_fato_completo):,} registros após join")

# Agora fazer merge com as dimensões Gold para obter os SRKs
print("  Fazendo merge com dimensões Gold...")

# Verificar colunas disponíveis nas dimensões Gold
print("  Verificando colunas das dimensões Gold...")
print(f"    DIM_MODELO tem srk_modelo? {'srk_modelo' in df_dim_modelo_gold_db.columns}")
print(f"    DIM_MODELO colunas: {list(df_dim_modelo_gold_db.columns)}")
print(f"    DIM_CONDICAO tem srk_condicao? {'srk_condicao' in df_dim_condicao_gold_db.columns}")
print(f"    DIM_CONDICAO colunas: {list(df_dim_condicao_gold_db.columns)}")
print(f"    DIM_COR tem srk_cor? {'srk_cor' in df_dim_cor_gold_db.columns}")
print(f"    DIM_COR colunas: {list(df_dim_cor_gold_db.columns)}")

# Verificar quais colunas de join existem em ambos os DataFrames
print(f"\n  Verificando colunas de join...")
print(f"    Colunas de join DIM_MODELO: {dim_modelo_cols}")
print(f"    Existem em df_fato_completo? {[c in df_fato_completo.columns for c in dim_modelo_cols]}")
print(f"    Existem em df_dim_modelo_gold_db? {[c in df_dim_modelo_gold_db.columns for c in dim_modelo_cols]}")

# Filtrar apenas colunas que existem em ambos os DataFrames
dim_modelo_cols_validas = [c for c in dim_modelo_cols if c in df_fato_completo.columns and c in df_dim_modelo_gold_db.columns]
print(f"    Colunas válidas para join: {dim_modelo_cols_validas}")

if len(dim_modelo_cols_validas) != len(dim_modelo_cols):
    print(f"    ⚠️  Aviso: {len(dim_modelo_cols) - len(dim_modelo_cols_validas)} colunas não encontradas em ambos DataFrames")
    print(f"       Colunas faltando: {set(dim_modelo_cols) - set(dim_modelo_cols_validas)}")

# Merge com DIM_MODELO (Gold)
df_fato_gold = df_fato_completo.merge(
    df_dim_modelo_gold_db,
    on=dim_modelo_cols_validas,
    how='left',
    suffixes=('', '_dim_modelo')
)

# Verificar se o SRK_modelo foi trazido (PostgreSQL retorna em minúsculas)
# O PostgreSQL retorna colunas em minúsculas, então procuramos por 'srk_modelo'
srk_modelo_col = 'srk_modelo' if 'srk_modelo' in df_fato_gold.columns else 'SRK_modelo'
print(f"    Colunas após merge DIM_MODELO: {[c for c in df_fato_gold.columns if 'srk' in c.lower() or 'modelo' in c.lower()]}")
if srk_modelo_col not in df_fato_gold.columns:
    print("  ⚠️  ERRO: srk_modelo não foi trazido no merge!")
    print(f"    Todas as colunas: {list(df_fato_gold.columns)[:15]}...")
    print(f"    Colunas de df_dim_modelo_gold_db: {list(df_dim_modelo_gold_db.columns)}")
    raise KeyError("srk_modelo não encontrado após merge com DIM_MODELO")
else:
    print(f"    ✓ {srk_modelo_col} encontrado! Valores não-nulos: {df_fato_gold[srk_modelo_col].notna().sum()}")
    # Renomear para maiúsculas para consistência
    if srk_modelo_col == 'srk_modelo':
        df_fato_gold = df_fato_gold.rename(columns={'srk_modelo': 'SRK_modelo'})

# Verificar colunas de condição
dim_condicao_cols_validas = [c for c in dim_condicao_cols if c in df_fato_gold.columns and c in df_dim_condicao_gold_db.columns]
print(f"    Colunas válidas para join DIM_CONDICAO: {dim_condicao_cols_validas}")

# Merge com DIM_CONDICAO (Gold)
df_fato_gold = df_fato_gold.merge(
    df_dim_condicao_gold_db,
    on=dim_condicao_cols_validas,
    how='left',
    suffixes=('', '_dim_condicao')
)

# Verificar se o SRK_condicao foi trazido (PostgreSQL retorna em minúsculas)
srk_condicao_col = 'srk_condicao' if 'srk_condicao' in df_fato_gold.columns else 'SRK_condicao'
if srk_condicao_col not in df_fato_gold.columns:
    print("  ⚠️  ERRO: srk_condicao não foi trazido no merge!")
    raise KeyError("srk_condicao não encontrado após merge com DIM_CONDICAO")
else:
    # Renomear para maiúsculas para consistência
    if srk_condicao_col == 'srk_condicao':
        df_fato_gold = df_fato_gold.rename(columns={'srk_condicao': 'SRK_condicao'})

# Verificar colunas de cor
dim_cor_cols_validas = [c for c in dim_cor_cols if c in df_fato_gold.columns and c in df_dim_cor_gold_db.columns]
print(f"    Colunas válidas para join DIM_COR: {dim_cor_cols_validas}")

# Merge com DIM_COR (Gold)
df_fato_gold = df_fato_gold.merge(
    df_dim_cor_gold_db,
    on=dim_cor_cols_validas,
    how='left',
    suffixes=('', '_dim_cor')
)

# Verificar se o SRK_cor foi trazido (PostgreSQL retorna em minúsculas)
srk_cor_col = 'srk_cor' if 'srk_cor' in df_fato_gold.columns else 'SRK_cor'
if srk_cor_col not in df_fato_gold.columns:
    print("  ⚠️  ERRO: srk_cor não foi trazido no merge!")
    raise KeyError("srk_cor não encontrado após merge com DIM_COR")
else:
    # Renomear para maiúsculas para consistência
    if srk_cor_col == 'srk_cor':
        df_fato_gold = df_fato_gold.rename(columns={'srk_cor': 'SRK_cor'})

print("  ✓ Todos os SRKs foram trazidos com sucesso!")

# Verificar se há registros sem match
missing_modelo = df_fato_gold['SRK_modelo'].isna().sum()
missing_condicao = df_fato_gold['SRK_condicao'].isna().sum()
missing_cor = df_fato_gold['SRK_cor'].isna().sum()

if missing_modelo > 0 or missing_condicao > 0 or missing_cor > 0:
    print(f"  ⚠️  Aviso: {missing_modelo} registros sem SRK_modelo")
    print(f"  ⚠️  Aviso: {missing_condicao} registros sem SRK_condicao")
    print(f"  ⚠️  Aviso: {missing_cor} registros sem SRK_cor")
    # Remover registros sem match
    df_fato_gold = df_fato_gold.dropna(subset=['SRK_modelo', 'SRK_condicao', 'SRK_cor'])

# Selecionar apenas colunas da tabela fato Gold
df_fato_gold_final = df_fato_gold[[
    'SRK_modelo',
    'SRK_condicao',
    'SRK_cor',
    'mileage',
    'price',
    'owner_count',
    'vehicle_age',
    'mileage_per_year',
    'brand_popularity'
]].copy()

print(f"  {len(df_fato_gold_final):,} registros finais")

print("\n  Tabela fato transformada com sucesso!")


Transformando tabela fato...
  Fazendo join com dimensões Silver...
  1,000,000 registros após join
  Fazendo merge com dimensões Gold...
  Verificando colunas das dimensões Gold...
    DIM_MODELO tem srk_modelo? True
    DIM_MODELO colunas: ['srk_modelo', 'make', 'model', 'year', 'trim', 'engine_hp', 'transmission', 'fuel_type', 'drivetrain', 'body_type']
    DIM_CONDICAO tem srk_condicao? True
    DIM_CONDICAO colunas: ['srk_condicao', 'seller_type', 'condition', 'accident_history']
    DIM_COR tem srk_cor? True
    DIM_COR colunas: ['srk_cor', 'exterior_color', 'interior_color']

  Verificando colunas de join...
    Colunas de join DIM_MODELO: ['make', 'model', 'year', 'trim', 'engine_hp', 'transmission', 'fuel_type', 'drivetrain', 'body_type']
    Existem em df_fato_completo? [True, True, True, True, True, True, True, True, True]
    Existem em df_dim_modelo_gold_db? [True, True, True, True, True, True, True, True, True]
    Colunas válidas para join: ['make', 'model', 'year', 'tri

## 8. Carregar Tabela Fato no Data Warehouse


In [15]:
print("Carregando tabela fato no Data Warehouse...")
print("="*60)

start_time = datetime.now()

# Carregar FATO_VEICULO
df_fato_gold_final.to_sql(
    'FATO_VEICULO',
    engine,
    schema='dw',
    if_exists='append',
    index=False,
    chunksize=10000
)

duration = (datetime.now() - start_time).total_seconds()
print(f"  {len(df_fato_gold_final):,} fatos carregados em {duration:.1f}s")


Carregando tabela fato no Data Warehouse...
  0 fatos carregados em 0.0s


## 9. Verificação e Validação


In [18]:
print("Verificando dados carregados...")
print("="*60)

try:
    with engine.connect() as conn:
        # Contar registros
        result = conn.execute(text("SELECT COUNT(*) FROM dw.DIM_MODELO"))
        dim_modelo_count = result.scalar()
        print(f"  dw.DIM_MODELO: {dim_modelo_count:,} registros")
        
        result = conn.execute(text("SELECT COUNT(*) FROM dw.DIM_CONDICAO"))
        dim_condicao_count = result.scalar()
        print(f"  dw.DIM_CONDICAO: {dim_condicao_count:,} registros")
        
        result = conn.execute(text("SELECT COUNT(*) FROM dw.DIM_COR"))
        dim_cor_count = result.scalar()
        print(f"  dw.DIM_COR: {dim_cor_count:,} registros")
        
        result = conn.execute(text("SELECT COUNT(*) FROM dw.FATO_VEICULO"))
        fato_count = result.scalar()
        print(f"  dw.FATO_VEICULO: {fato_count:,} registros")
        
        # Verificar se há dados antes de calcular estatísticas
        if fato_count > 0:
            # Estatísticas de preço
            print("\n" + "-"*60)
            print("ESTATÍSTICAS DE PREÇO (Gold):")
            print("-"*60)
            
            result = conn.execute(text("""
                SELECT 
                    ROUND(AVG(price)::numeric, 2) as preco_medio,
                    ROUND(MIN(price)::numeric, 2) as preco_minimo,
                    ROUND(MAX(price)::numeric, 2) as preco_maximo
                FROM dw.FATO_VEICULO
            """))
            stats = result.fetchone()
            
            if stats and stats[0] is not None:
                print(f"Preço Médio:  ${stats[0]:,.2f}")
                print(f"Preço Mínimo: ${stats[1]:,.2f}")
                print(f"Preço Máximo: ${stats[2]:,.2f}")
            else:
                print("  Nenhuma estatística disponível")
            
            # Top 5 marcas
            print("\n" + "-"*60)
            print("TOP 5 MARCAS (por volume):")
            print("-"*60)
            
            # Usar aspas duplas para preservar maiúsculas ou minúsculas conforme o banco
            # PostgreSQL retorna em minúsculas, então usamos minúsculas na consulta
            result = conn.execute(text("""
                SELECT m.make, COUNT(*) as total
                FROM dw.FATO_VEICULO f
                JOIN dw.DIM_MODELO m ON f.srk_modelo = m.srk_modelo
                GROUP BY m.make
                ORDER BY total DESC
                LIMIT 5
            """))
            
            rows = result.fetchall()
            if rows:
                for row in rows:
                    print(f"  {row[0]:15s}: {row[1]:,}")
            else:
                print("  Nenhuma marca encontrada")
        else:
            print("\n  ⚠️  AVISO: Tabelas Gold estão vazias!")
            print("  Execute as células anteriores para carregar os dados.")
    
    print("\n" + "="*60)
    print("  ETL Silver → Gold CONCLUÍDO COM SUCESSO!")
    print("="*60)
    
except Exception as e:
    print(f"  Erro na verificação: {e}")
    raise


Verificando dados carregados...
  dw.DIM_MODELO: 0 registros
  dw.DIM_CONDICAO: 0 registros
  dw.DIM_COR: 0 registros
  dw.FATO_VEICULO: 0 registros

  ⚠️  AVISO: Tabelas Gold estão vazias!
  Execute as células anteriores para carregar os dados.

  ETL Silver → Gold CONCLUÍDO COM SUCESSO!
